In [4]:
import configparser
import glob,os,sys
import time
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
from astropy.table import Table
from astropy.table import QTable
from photutils import aperture_photometry
from astropy.coordinates import SkyCoord  # High-level coordinates
import astropy.units as u
from photutils import SkyCircularAperture
import pyregion

from HST_filters import HST_filt  # This will point to the HST throughput files

In [5]:
####################################
# constants
####################################
c = 2.9979E10       # cm/s
Ang = 1E-8          # cm
Jy = 1.0E-23        # erg/s/cm^2/Hz
mJy = 1e-3          # Jy
uJy = 1e-6          # Jy
Mpc = 3.086e24      # cm
## PHOTFLAM: inverse sensitivity (erg * cm**(-2) * s**(-1) Angstrom**(-1)
## PHOTPLAM: Pivot wavelength

In [ ]:
####################################
# variables and flags
####################################
cfg = "/Users/brianmerino/Desktop/Apperture_Test/Code/clump.cfg"
bkgd_subtract = 1   # use background subtracted frames for photometry
ext = 0
####################################

# read-in cfg file with paramters 
config = configparser.ConfigParser()
config.read(cfg)
img_dir = config.get('CONFIG','img_dir')
sub_img_dir_drz = config.get('CONFIG','sub_img_dir_drz')
#sub_img_dir_wht = config.get('CONFIG','sub_img_dir_wht')
plot_dir = config.get('CONFIG','plot_dir')
catalog_file = config.get('CONFIG','catalog_file')
hst_filt = config.get('CONFIG','hst_filt')

In [ ]:
def hst_phot(photflam,photplam,dn,dn_err=None,unit="Jy"):
    ## http://www.stsci.edu/hst/acs/analysis/zeropoints
    #ABMAG_ZP=-2.5*np.log10(photflam)-5*np.log10(photplam)-2.408 
    #m = -2.5*np.log10(dn) + ABMAG_ZP
    C = photflam/Ang*(photplam*Ang)**2/c/Jy   # Jy
    if unit == "Jy": Fnu = dn*C
    elif unit == "mJy": Fnu = dn*C/mJy
    elif unit == "uJy": Fnu = dn*C/uJy
    ABmag = -2.5*np.log10(Fnu*Jy) - 48.60
    if dn_err:
        if unit == "Jy": Fnu_err = dn_err*C
        elif unit == "mJy": Fnu_err = dn_err*C/mJy
        elif unit == "uJy": Fnu_err = dn_err*C/uJy
        ABmag_err = 2.5*(Fnu_err/(Fnu*np.log(10)))
        return ABmag,ABmag_err
    else:
        return ABmag

In [ ]:
def phot(image_drz,noise,photflam,photplam):
    flux = np.sum(image_drz)
    
    flux_err = noise
    print(flux_err)
    
    if noise == 0:
        SN = 0
    else:
        SN = flux/noise
    
    if item == 'f225w' or item == 'f275w' or item == 'f336w' or item == 'f390w':
        if drop_uvis:
            mag = -99
            magerr = -99
        
        elif SN < 8:
            mag = -99
            magerr= -99
        
        else:
            mag, magerr = hst_phot(photflam, photplam, flux, dn_err=flux_err) # AB magnitude
    
    else:
        if SN < 3:
            mag = -99
            magerr = -99
    
        else:
            mag, magerr = hst_phot(photflam, photplam, flux, dn_err=flux_err) # AB magnitude
    
    return round(mag,4),round(magerr,4)

In [ ]:
bkgd_subtract = 1   # use background subtracted frames for photometry

img_dir = config.get('CONFIG','img_dir')
sub_img_dir_drz = config.get('CONFIG','sub_img_dir_drz')
plot_dir = config.get('CONFIG','plot_dir')

hst_filt = config.get('CONFIG','hst_filt')

In [6]:
catalog_file = '/Users/brian.merino/Google Drive/My Drive/Clumpy_Research/galaxy_phot_v3.cat'
catalog = Table.read(catalog_file,format='ascii')

catalog.show_in_notebook()

idx,cluster,id,RA,DEC,z_clash,z_phot,z_16,z_84,f225w_mag,f225w_magerr,f275w_mag,f275w_magerr,f336w_mag,f336w_magerr,f390w_mag,f390w_magerr,f435w_mag,f435w_magerr,f475w_mag,f475w_magerr,f606w_mag,f606w_magerr,f625w_mag,f625w_magerr,f775w_mag,f775w_magerr,f814w_mag,f814w_magerr,f850lp_mag,f850lp_magerr,f105w_mag,f105w_magerr,f110w_mag,f110w_magerr,f125w_mag,f125w_magerr,f140w_mag,f140w_magerr,f160w_mag,f160w_magerr,f225w_flux,f225w_fluxerr,f275w_flux,f275w_fluxerr,f336w_flux,f336w_fluxerr,f390w_flux,f390w_fluxerr,f435w_flux,f435w_fluxerr,f475w_flux,f475w_fluxerr,f606w_flux,f606w_fluxerr,f625w_flux,f625w_fluxerr,f775w_flux,f775w_fluxerr,f814w_flux,f814w_fluxerr,f850lp_flux,f850lp_fluxerr,f105w_flux,f105w_fluxerr,f110w_flux,f110w_fluxerr,f125w_flux,f125w_fluxerr,f140w_flux,f140w_fluxerr,f160w_flux,f160w_fluxerr,z_spec
0,a209,218,22.9730375,-13.5973175,0.939,0.8484974503517151,0.8219921588897705,0.8998340964317322,21.022,0.025,-99.0,-99.0,22.711,0.068,24.107,0.13,24.274,0.114,24.094,0.068,23.743,0.033,23.941,0.068,22.948,0.034,22.67,0.02,22.769,0.06,22.679,0.015,22.58,0.008,22.393,0.012,22.3,0.009,22.298,0.014,14.164459272385136,0.3261487277091531,-99.0,-99.0,2.989509927814587,0.18723394706790525,0.826418441715795,0.09895073679244823,0.7085981269167676,0.07440130126866958,0.8363729915840239,0.05238230352683446,1.1555799554021289,0.035122839563746,0.9629417129715555,0.06030934235692439,2.4032558049332398,0.07525833347883332,3.104559588128347,0.057188101023488536,2.834001005509097,0.15661268325156918,3.078931309741611,0.042537008016982,3.37287308658869,0.024852247327167787,4.006821541822104,0.04428502825193758,4.365158322401652,0.03618413453419539,4.373206651696452,0.05639021049076047,0.939
1,a209,1020,22.978324999999998,-13.612234166666667,0.488,0.5941091179847717,0.5152083039283752,0.6043280363082886,-99.0,-99.0,-99.0,-99.0,21.628,0.032,22.892,0.055,22.664,0.034,22.573,0.022,21.845,0.007,21.672,0.011,21.32,0.01,21.218,0.007,21.196,0.018,21.105,0.005,21.089,0.003,21.036,0.004,20.947,0.003,20.869,0.005,-99.0,-99.0,-99.0,-99.0,8.105876824647362,0.23890523061861027,2.5304630494613947,0.1281853429133654,3.121763510848157,0.09775851528547144,3.3946890538007795,0.06878573161355929,6.637430704019088,0.04279309718559492,7.783949326267247,0.07886210500644963,10.764652136298333,0.09914611016122821,11.824968703831036,0.07623839065517492,12.067019095932718,0.200054435667618,13.121998990192008,0.060429038530198084,13.316803762699811,0.036795688596383526,13.982988484658337,0.051515233344450705,15.177491558612486,0.041936958974284885,16.30797361023593,0.0751009938637391,0.488
2,a209,1312,22.97045,-13.616914166666668,0.495,0.5900025367736816,0.568169355392456,0.6054204106330872,-99.0,-99.0,-99.0,-99.0,21.793,0.036,22.677,0.042,22.647,0.031,22.343,0.016,21.631,0.006,21.464,0.008,21.116,0.008,21.062,0.006,21.001,0.014,20.965,0.004,20.951,0.002,20.896,0.004,20.831,0.003,20.775,0.004,-99.0,-99.0,-99.0,-99.0,6.963057796029312,0.23087567639222017,3.0846081377571397,0.1193232216242256,3.1710274389478936,0.09053935032891668,4.195657223387249,0.061829489778459326,8.083510373900667,0.04467112916641537,9.427575085274157,0.0694649403341899,12.989726288978538,0.09571184036823951,13.652116950664434,0.07544438634818701,14.441090844161184,0.18621030682429474,14.927944095789943,0.05499657847042305,15.121678487917649,0.027855161173862338,15.907429237502162,0.05860513508820819,16.88884694790917,0.04666560866413637,17.78279410038923,0.06551423457174192,0.495
3,a209,1602,22.96975,-13.621598888888888,0.547,0.395388662815094,0.31781673431396484,0.39996740221977234,-99.0,-99.0,-99.0,-99.0,22.891,0.087,23.128,0.057,23.884,0.086,22.976,0.026,22.402,0.01,22.288,0.016,22.081,0.016,22.072,0.013,22.152,0.037,22.227,0.011,22.268,0.007,22.304,0.012,22.277,0.01,22.276,0.015,-99.0,-99.0,-99.0,-99.0,2.5327947656899945,0.20295274639397126,2.036104201131967,0.10689331253384562,1.0148456628180937,0.0803848362243744,2.3420706947938243,0.05608529751313286,3.973745050614623,0.036599544467

In [7]:
filter_list = ['f775w','f814w']

In [9]:
for i in range(len(catalog[:8])):
    for f in filter_list:
        #print(catalog['id'][i])
        cluster = catalog['cluster'][i]
        ID = catalog['Id'][i]
        image_path = '%s-%s-%s_galaxy.fits'%(f,cluster,ID)
        wht_path   = '%s-%s-%s_galaxy_wht.fits'%(f,cluster,ID)

        pf = fits.open(image_path)
        image    = pf[ext].data
        head     = pf[ext].header
        photflam = head['photflam']
        photplam = head['photplam']
        photbw   = head['photbw']

        pf_wht = fits.open(wht_path)
        wht = pf_wht[ext].data #inverse variance weight images

        quotient = 1/wht
        
        
        
        for i in range(0,quotient.shape[0]):
            for j in range(0,quotient.shape[1]):
                if np.isinf(quotient[i][j]):
                    quotient[i][j]=0

        SUM = np.sum(quotient)

        noise = np.sqrt(SUM)

        wcs = WCS(head)
        wcs.sip = None

        mag[item+'_mag'],magerr[item+'_magerr']=phot(image,noise,photflam,photplam)
        pivot[item]=float(photplam)
        bandwidth[item]=float(photbw)/2

        #test_magerr,test_magerr = phot(image_drz,noise,photflam,photplam)

218
1020
1312
1602
1669
1671
1708
1772


In [ ]:
path = '/Users/brianmerino/Desktop/Apperture_Test/%s-%s/galaxy_phot.csv'%(cluster,ID)

if os.path.exists(path):
    os.remove(path)

In [ ]:
# This cfg file will contain the variables that each code needs. 
cfg = "/Apperture_Test/Code/general.cfg"

config = configparser.ConfigParser()
config.read(cfg)

cluster = config.get('CONFIG','cluster')
ID = int(config.get('CONFIG','Id'))
z = float(config.get('CONFIG','z'))

counter = -1 #In the region files, the galactic coords are listed last
drop_uvis = 1
mag,magerr = {},{}
pivot = {}
bandwidth = {}

path = '/Users/brianmerino/Desktop/Apperture_Test/%s-%s/galaxy_phot.csv'%(cluster,ID)

if os.path.exists(path):
    os.remove(path)
    
    # Set up header for catalog
    print('#1 \t cluster \n#2 \t id \n#3 \t obj \n#4\t RA \n#5 \t DEC \n#6 \t area \n\
#7 \t zb \n#8 \t f225w_mag \n#9 \t f225w_magerr \n#10 \t f275w_mag \n\
#11 \t f275w_magerr \n#12 \t f336w_mag \n#13 \t f336w_magerr \n#14 \t f390w_mag \n\
#15 \t f390w_magerr \n#16 \t f435w_mag \n#17 \t f435w_magerr \n#18 \t f475w_mag \n\
#19 \t f475w_magerr \n#20 \t f606w_mag \n#21 \t f606w_magerr \n#22 \t f625w_mag \n\
#23 \t f625w_magerr \n#24 \t f775w_mag \n#25 \t f775w_magerr \n#26 \t f814w_mag \n\
#27 \t f814w_magerr \n#28 \t f850lp_mag \n#29 \t f850lp_magerr \n#30 \t f105w_mag \n\
#31 \t f105w_magerr \n#32 \t f110w_mag \n#33 \t f110w_magerr \n#34 \t f125w_mag \n\
#35 \t f125w_magerr \n#36 \t f140w_mag \n#37 \t f140w_magerr \n#38 \t f160w_mag \n\
#39 \t f160w_magerr', file=open(path, "a"))

else:
    # Set up header for catalog
    print('#1 \t cluster \n#2 \t id \n#3 \t obj \n#4\t RA \n#5 \t DEC \n#6 \t area \n\
#7 \t zb \n#8 \t f225w_mag \n#9 \t f225w_magerr \n#10 \t f275w_mag \n\
#11 \t f275w_magerr \n#12 \t f336w_mag \n#13 \t f336w_magerr \n#14 \t f390w_mag \n\
#15 \t f390w_magerr \n#16 \t f435w_mag \n#17 \t f435w_magerr \n#18 \t f475w_mag \n\
#19 \t f475w_magerr \n#20 \t f606w_mag \n#21 \t f606w_magerr \n#22 \t f625w_mag \n\
#23 \t f625w_magerr \n#24 \t f775w_mag \n#25 \t f775w_magerr \n#26 \t f814w_mag \n\
#27 \t f814w_magerr \n#28 \t f850lp_mag \n#29 \t f850lp_magerr \n#30 \t f105w_mag \n\
#31 \t f105w_magerr \n#32 \t f110w_mag \n#33 \t f110w_magerr \n#34 \t f125w_mag \n\
#35 \t f125w_magerr \n#36 \t f140w_mag \n#37 \t f140w_magerr \n#38 \t f160w_mag \n\
#39 \t f160w_magerr', file=open(path, "a"))






if cluster == 'a611' or cluster =='macs0717' or cluster =='macs0744':
    filter_list = ['f105w', 'f110w', 'f125w', 'f140w', 'f160w',\
               'f225w', 'f275w', 'f336w', 'f390w', 'f435w',\
               'f475w', 'f606w', 'f775w', 'f814w', 'f850lp']

elif cluster == 'macs1423':
    filter_list = ['f105w', 'f110w', 'f125w', 'f140w', 'f160w', 'f225w', 'f275w', 'f336w',\
                   'f390w', 'f435w', 'f475w', 'f606w', 'f775w', 'f850lp']

else:
    filter_list = ['f105w', 'f110w', 'f125w', 'f140w', 'f160w',\
               'f225w', 'f275w', 'f336w', 'f390w', 'f435w',\
               'f475w', 'f606w', 'f625w', 'f775w', 'f814w', 'f850lp']



region_path = '/Users/brianmerino/Desktop/Individual_Clumps/%s/'%(cluster)
region_name = region_path+'%s.reg'%(ID)
r = pyregion.open(region_name)


for item in filter_list:
    pf = fits.open(sub_img_dir_drz+'%s-%s/masked/galaxy_'%(cluster,ID)+item+'.fits')
    image = pf[ext].data
    head = pf[ext].header
    photflam = head['photflam']
    photplam = head['photplam']
    photbw   = head['photbw']
    
    pf_wht = fits.open(sub_img_dir_drz+'%s-%s/masked/galaxy_'%(cluster,ID)+item+'_wht.fits')
    #  inverse variance weight images
    wht = pf_wht[ext].data
    
    quotient = 1/wht
    
    for i in range(0,quotient.shape[0]):
        for j in range(0,quotient.shape[1]):
            if np.isinf(quotient[i][j]):
                quotient[i][j]=0
    
    SUM = np.sum(quotient)
    
    noise = np.sqrt(SUM)
    
    wcs = WCS(head)
    wcs.sip = None
    
    mag[item+'_mag'],magerr[item+'_magerr']=phot(image,noise,photflam,photplam)
    pivot[item]=float(photplam)
    bandwidth[item]=float(photbw)/2
    
    #counter +=1


fig = plt.figure(figsize=(8,6))
p1 = fig.add_subplot(111)


# Merino photometry (from dictionary)
mag1 = [mag[obj] for obj in mag]
magerr1 = [magerr[obj] for obj in magerr]
pivots = [pivot[obj] for obj in pivot]
bandwidths = [bandwidth[obj] for obj in bandwidth]



# determine range for plotting (filtering bad mags)
mag1 = np.array(mag1)

for i in range(0,len(mag1)):
    if np.isinf(mag1[i]):
        mag1[i]=-99

#mask = mag2 > -99
mask = mag1 > -50
mag1_mask = np.compress(mask,mag1,0)
y0 = np.min(mag1_mask)
y1 = np.max(mag1_mask)
#print mag2
#print mag2_mask
#print y0
#print y1


# This section will direct the code to the HST filter throughputs files
x_f105w, y_f105w, x_f110w, y_f110w, x_f125w, y_f125w, x_f140w, y_f140w,\
x_f160w, y_f160w, x_f225w, y_f225w, x_f275w, y_f275w, x_f336w, y_f336w,\
x_f390w, y_f390w, x_f435w, y_f435w, x_f475w, y_f475w, x_f606w, y_f606w,\
x_f625w, y_f625w, x_f775w, y_f775w, x_f814w, y_f814w, x_f850lp, y_f850lp = HST_filt()



p1.errorbar(pivots, mag1, xerr=bandwidths, yerr=magerr1, marker='o', linestyle='', label="Merino", zorder=2)
#p1.set_title('CLASH id: ' + str(cluster)+  ' ' + str(ID) + ' Clump '+str(counter)+', z = ' + str(z))
p1.set_title('CLASH id: ' + str(cluster)+  ' ' + str(ID) + ' Galaxy,  z = ' + str(z))
p1.legend(loc=2)

p1.set_xlabel('Wavelength [Angstrom]')
p1.set_ylabel('Magnitude [AB]')
p1.set_ylim(y0 - 0.75,y1 + 0.75)
p1.invert_yaxis()

alpha = 0.4
p2 = p1.twinx()
p2.plot(x_f105w, y_f105w, zorder=1, label = 'F105W', c = 'green', alpha = alpha)
p2.plot(x_f110w, y_f110w, zorder=1, label = 'F110W', c = 'green', alpha = alpha)
p2.plot(x_f125w, y_f125w, zorder=1, label = 'F125W', c = 'green', alpha = alpha)
p2.plot(x_f140w, y_f140w, zorder=1, label = 'F140W', c = 'green', alpha = alpha)
p2.plot(x_f160w, y_f160w, zorder=1, label = 'F160W', c = 'green', alpha = alpha)
p2.plot(x_f225w, y_f225w, zorder=1, label = 'F225W', c = 'green', alpha = alpha)
p2.plot(x_f275w, y_f275w, zorder=1, label = 'F275W', c = 'green', alpha = alpha)
p2.plot(x_f336w, y_f336w, zorder=1, label = 'F336W', c = 'green', alpha = alpha)
p2.plot(x_f390w, y_f390w, zorder=1, label = 'F390W', c = 'blue', alpha = alpha)
p2.plot(x_f435w, y_f435w, zorder=1, label = 'F435W', c = 'blue', alpha = alpha)
p2.plot(x_f475w, y_f475w, zorder=1, label = 'F475W', c = 'blue', alpha = alpha)
p2.plot(x_f606w, y_f606w, zorder=1, label = 'F606W', c = 'blue', alpha = alpha)
p2.plot(x_f625w, y_f625w, zorder=1, label = 'F625W', c = 'blue', alpha = alpha)
p2.plot(x_f775w, y_f775w, zorder=1, label = 'F775W', c = 'blue', alpha = alpha)
p2.plot(x_f814w, y_f814w, zorder=1, label = 'F814W', c = 'blue', alpha = alpha)
p2.plot(x_f850lp, y_f850lp, zorder=1, label = 'F850lp', c = 'blue', alpha = alpha)
p2.set_xlim(1000, 17000)
p2.set_ylim(0,0.6)


fontsize = 8
p2.annotate('F105W', xy=(10500,0.02), xytext=(10000,0.06), fontsize=fontsize)
p2.annotate('F110W', xy=(11000,0.02), xytext=(10500,0.02), fontsize=fontsize)
p2.annotate('F125W', xy=(12500,0.02), xytext=(12000,0.04), fontsize=fontsize)
p2.annotate('F140W', xy=(14000,0.02), xytext=(13500,0.06), fontsize=fontsize)
p2.annotate('F160W', xy=(16000,0.02), xytext=(15500,0.02), fontsize=fontsize)
p2.annotate('F225W', xy=(2250,0.02), xytext=(1750,0.02), fontsize=fontsize)
p2.annotate('F275W', xy=(2750,0.02), xytext=(2250,0.04), fontsize=fontsize)
p2.annotate('F336W', xy=(3360,0.02), xytext=(2960,0.06), fontsize=fontsize)
p2.annotate('F390W', xy=(3900,0.02), xytext=(3400,0.02), fontsize=fontsize)
p2.annotate('F435W', xy=(4350,0.02), xytext=(3950,0.04), fontsize=fontsize)
p2.annotate('F475W', xy=(4750,0.02), xytext=(4450,0.06), fontsize=fontsize)
p2.annotate('F606W', xy=(6060,0.02), xytext=(5560,0.02), fontsize=fontsize)
p2.annotate('F625W', xy=(6250,0.02), xytext=(5750,0.04), fontsize=fontsize)
p2.annotate('F775W', xy=(7750,0.02), xytext=(7250,0.06), fontsize=fontsize)
p2.annotate('F814W', xy=(8140,0.02), xytext=(7640,0.02), fontsize=fontsize)
p2.annotate('F850lp', xy=(8500,0.02), xytext=(8000,0.04), fontsize=fontsize)



# This section will display approximately where the emission lines H-alpha
# and [OIII] should be after accounting for the source's redshift.
p3 = p1.twinx()
#z = 0.579

H_alpha = (1+z)*6563  #Angstroms
H_beta  = (1+z)*4861.3  #Angstroms
OII     = (1+z)*3727  #Angstroms
OIII    = (1+z)*5007  #Angstroms

alpha1 = 0.5
p3.axvline(H_alpha, c='black', alpha=alpha1)
p3.axvline(OIII, c='black', alpha=alpha1)
p3.axvline(H_beta, c='black', alpha=alpha1)
p3.axvline(OII, c='black', alpha=alpha1)
plt.ylim(0,0.6)

p3.annotate('H-$\\alpha$', xy=(H_alpha, 0.56), xytext=(H_alpha, 0.57), rotation=90)
p3.annotate('H-$\\beta$', xy=(H_beta, 0.56), xytext=(H_beta-400, 0.57), rotation=90)
p3.annotate('[OII]', xy=(OII, 0.56), xytext=(OII, 0.565), rotation=90)
p3.annotate('[OIII]', xy=(OIII, 0.56), xytext=(OIII, 0.56), rotation=90)


#plt.savefig(plot_dir + "/%s_%s.png" % (cluster,obj))
plt.savefig(plot_dir + '/%s-%s/galaxy.png'%(cluster,ID))
plt.show()


# This section establishes the name of each column for the catalog
Cluster, CLASHID, RA, DEC, area, redshift = [],[],[],[],[],[]

f105w_mag,f110w_mag,f125w_mag,f140w_mag,f160w_mag = [],[],[],[],[]
f225w_mag,f275w_mag,f336w_mag,f390w_mag,f435w_mag = [],[],[],[],[]
f475w_mag,f606w_mag,f625w_mag,f775w_mag,f814w_mag,f850lp_mag = [],[],[],[],[],[]

f105w_err,f110w_err,f125w_err,f140w_err,f160w_err = [],[],[],[],[]
f225w_err,f275w_err,f336w_err,f390w_err,f435w_err = [],[],[],[],[]
f475w_err,f606w_err,f625w_err,f775w_err,f814w_err,f850lp_err = [],[],[],[],[],[]

Cluster.append(cluster)
CLASHID.append(ID)
RA.append(r[counter].coord_list[0])
DEC.append(r[counter].coord_list[1])
area.append(Area)
redshift.append(z)
f105w_mag.append(round(mag['f105w_mag'],3))
f110w_mag.append(round(mag['f110w_mag'],3))
f125w_mag.append(round(mag['f125w_mag'],3))
f140w_mag.append(round(mag['f140w_mag'],3))
f160w_mag.append(round(mag['f160w_mag'],3))
f225w_mag.append(round(mag['f225w_mag'],3))
f275w_mag.append(round(mag['f275w_mag'],3))
f336w_mag.append(round(mag['f336w_mag'],3))
f390w_mag.append(round(mag['f390w_mag'],3))
f435w_mag.append(round(mag['f435w_mag'],3))
f475w_mag.append(round(mag['f475w_mag'],3))
f606w_mag.append(round(mag['f606w_mag'],3))

if cluster=='a611' or cluster=='macs0717' or cluster=='macs1423' or cluster =='macs0744':
    f625w_mag.append(-99)
else:
    f625w_mag.append(round(mag['f625w_mag'],3))

f775w_mag.append(round(mag['f775w_mag'],3))

if cluster=='macs1423':
    f814w_mag.append(-99)
else:
    f814w_mag.append(round(mag['f814w_mag'],3))

f850lp_mag.append(round(mag['f850lp_mag'],3))


f105w_err.append(round(magerr['f105w_magerr'],3))
f110w_err.append(round(magerr['f110w_magerr'],3))
f125w_err.append(round(magerr['f125w_magerr'],3))
f140w_err.append(round(magerr['f140w_magerr'],3))
f160w_err.append(round(magerr['f160w_magerr'],3))
f225w_err.append(round(magerr['f225w_magerr'],3))
f275w_err.append(round(magerr['f275w_magerr'],3))
f336w_err.append(round(magerr['f336w_magerr'],3))
f390w_err.append(round(magerr['f390w_magerr'],3))
f435w_err.append(round(magerr['f435w_magerr'],3))
f475w_err.append(round(magerr['f475w_magerr'],3))
f606w_err.append(round(magerr['f606w_magerr'],3))

if cluster=='a611' or cluster=='macs0717' or cluster=='macs1423' or cluster =='macs0744':
    f625w_err.append(-99)
else:
    f625w_err.append(round(magerr['f625w_magerr'],3))

f775w_err.append(round(magerr['f775w_magerr'],3))

if cluster=='macs1423':
    f814w_err.append(-99)
else:
    f814w_err.append(round(magerr['f814w_magerr'],3))

f850lp_err.append(round(magerr['f850lp_magerr'],3))



print(Cluster[0],' ',CLASHID[0],' ',counter,' ',RA[0],' ',DEC[0],' ',area[0],' ',redshift[0],' ',\
      f225w_mag[0],' ',f225w_err[0],' ',f275w_mag[0],' ',f275w_err[0],' ',f336w_mag[0],' ',f336w_err[0],' ',f390w_mag[0],' ',f390w_err[0],' ',\
      f435w_mag[0],' ',f435w_err[0],' ',f475w_mag[0],' ',f475w_err[0],' ',f606w_mag[0],' ',f606w_err[0],' ',\
      f625w_mag[0],' ',f625w_err[0],' ',f775w_mag[0],' ',f775w_err[0],' ',f814w_mag[0],' ',f814w_err[0],' ',\
      f850lp_mag[0],' ',f850lp_err[0],' ',f105w_mag[0],' ',f105w_err[0],' ',f110w_mag[0],' ',f110w_err[0],' ',\
      f125w_mag[0],' ',f125w_err[0],' ',f140w_mag[0],' ',f140w_err[0],' ',f160w_mag[0],' ',f160w_err[0], file=open(path, "a"))